<!-- <p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/images/IDSN-logo.png" width="200" alt="Skills Network Logo">
    </a>
</p> -->


# **Getting Started With Spark using Python**


![](http://spark.apache.org/images/spark-logo.png)


### The Python API


Spark is written in Scala, which compiles to Java bytecode, but you can write python code to communicate to the java virtual machine through a library called py4j. Python has the richest API, but it can be somewhat limiting if you need to use a method that is not available, or if you need to write a specialized piece of code. The latency associated with communicating back and forth to the JVM can sometimes cause the code to run slower.
An exception to this is the SparkSQL library, which has an execution planning engine that precompiles the queries. Even with this optimization, there are cases where the code may run slower than the native scala version.
The general recommendation for PySpark code is to use the "out of the box" methods available as much as possible and avoid overly frequent (iterative) calls to Spark methods. If you need to write high-performance or specialized code, try doing it in scala.
But hey, we know Python rules, and the plotting libraries are way better. So, it's up to you!


## Objectives


In this Notebook, we will go over the basics of Apache Spark and PySpark. We will start with creating the SparkContext and SparkSession. We then create an RDD and apply some basic transformations and actions. Finally we demonstrate the basics dataframes and SparkSQL.

After this Notebook you will be able to:

* Create the SparkContext and SparkSession
* Create an RDD and apply some basic transformations and actions to RDDs
* Demonstrate the use of the basics Dataframes and SparkSQL


----


## Setup


For this lab, we are going to be using Python and Spark (PySpark). These libraries should be installed in your lab environment or in SN Labs.


In [20]:
# Installing required packages
!pip install pyspark
!pip install findspark

In [21]:
import findspark
findspark.init()

In [22]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context. 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

## Spark Context and Spark Session


In this exercise, you will create the Spark Context and initialize the Spark session needed for SparkSQL and DataFrames.
SparkContext is the entry point for Spark applications and contains functions to create RDDs such as `parallelize()`. SparkSession is needed for SparkSQL and DataFrame operations.


####  Creating the spark session and context


In [ ]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

####  Initialize Spark session
To work with dataframes we just need to verify that the spark session instance has been created.


In [ ]:
spark

##  RDDs
In this exercise we work with Resilient Distributed Datasets (RDDs). RDDs are Spark's primitive data abstraction and we use concepts from functional programming to create and manipulate RDDs. 


####  Create an RDD.
For demonstration purposes, we create an RDD here by calling `sc.parallelize()`  
We create an RDD which has integers from 1 to 30.


In [ ]:
data = range(1,100)
# print first element of iterator
print("length of Data is",len(data))
xrangeRDD = sc.parallelize(data, len(data))

# this will let us know that we created an RDD
xrangeRDD

length of Data is 99


PythonRDD[1] at RDD at PythonRDD.scala:53

####  Transformations


A transformation is an operation on an RDD that results in a new RDD. The transformed RDD is generated rapidly because the new RDD is lazily evaluated, which means that the calculation is not carried out when the new RDD is generated. The RDD will contain a series of transformations, or computation instructions, that will only be carried out when an action is called. In this transformation, we reduce each element in the RDD by 1. Note the use of the lambda function. We also then filter the RDD to only contain elements <10.


In [ ]:
subRDD = xrangeRDD.map(lambda x: x-1)
filteredRDD = subRDD.filter(lambda x : x<50)

####  Actions 


A transformation returns a result to the driver. We now apply the `collect()` action to get the output from the transformation.


In [ ]:
print(filteredRDD.collect())
filteredRDD.count()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


50

####  Caching Data


This simple example shows how to create an RDD and cache it. Notice the **10x speed improvement**!  If you wish to see the actual computation time, browse to the Spark UI...it's at `localhost:4040`.  You'll see that the second calculation took much less time!


In [ ]:
import time 

test = sc.parallelize(range(1,50000),4)
test.cache()

t1 = time.time()
# first count will trigger evaluation of count *and* cache
count1 = test.count()
dt1 = time.time() - t1
print("dt1: ", dt1)


t2 = time.time()
# second count operates on cached data only
count2 = test.count()
dt2 = time.time() - t2
print("dt2: ", dt2)

#test.count()

dt1:  0.6610550880432129
dt2:  0.3174583911895752


##  DataFrames and SparkSQL


In order to work with the extremely powerful SQL engine in Apache Spark, you will need a Spark Session. We have created that in the first Exercise, let us verify that spark session is still active.


In [ ]:
spark

####  Create Your First DataFrame!


You can create a structured data set (much like a database table) in Spark.  Once you have done that, you can then use powerful SQL tools to query and join your dataframes.


In [ ]:
# Spark SQL provides spark.read().csv("file_name") to read a file or directory of files in CSV format into Spark DataFrame.
df = spark.read.option("header", True).csv("Datasets/us-500.csv").cache()

In [24]:
# Print the dataframe as well as the data schema
df.show()
df.printSchema()

+----------+---------+--------------------+--------------------+-------------+--------------+-----+-----+------------+------------+--------------------+--------------------+
|first_name|last_name|        company_name|             address|         city|        county|state|  zip|      phone1|      phone2|               email|                 web|
+----------+---------+--------------------+--------------------+-------------+--------------+-----+-----+------------+------------+--------------------+--------------------+
|     James|     Butt|   Benton, John B Jr|  6649 N Blue Gum St|  New Orleans|       Orleans|   LA|70116|504-621-8927|504-845-1427|     jbutt@gmail.com|http://www.benton...|
| Josephine|  Darakjy|Chanay, Jeffrey A...| 4 B Blue Ridge Blvd|     Brighton|    Livingston|   MI|48116|810-292-9388|810-374-9840|josephine_darakjy...|http://www.chanay...|
|       Art|   Venere| Chemel, James L Cpa|8 W Cerritos Ave #54|   Bridgeport|    Gloucester|   NJ|08014|856-636-8749|856-264-4130

In [ ]:
# Register the DataFrame as a SQL temporary view
df.createTempView("us_500")

####  Explore the data using DataFrame functions and SparkSQL

In this section, we explore the datasets using functions both from dataframes as well as corresponding SQL queries using sparksql. Note the different ways to achieve the same task!


In [ ]:
# Select and show basic data columns

df.select("first_name").show() 
# or
df.select(df["first_name"]).show()


+----------+
|first_name|
+----------+
|     James|
| Josephine|
|       Art|
|     Lenna|
|   Donette|
|    Simona|
|    Mitsue|
|     Leota|
|      Sage|
|      Kris|
|     Minna|
|      Abel|
|     Kiley|
|  Graciela|
|     Cammy|
|    Mattie|
|   Meaghan|
|    Gladys|
|      Yuki|
|  Fletcher|
+----------+
only showing top 20 rows

+----------+
|first_name|
+----------+
|     James|
| Josephine|
|       Art|
|     Lenna|
|   Donette|
|    Simona|
|    Mitsue|
|     Leota|
|      Sage|
|      Kris|
|     Minna|
|      Abel|
|     Kiley|
|  Graciela|
|     Cammy|
|    Mattie|
|   Meaghan|
|    Gladys|
|      Yuki|
|  Fletcher|
+----------+
only showing top 20 rows



In [ ]:
# Select and show basic data columns using sql

spark.sql("SELECT first_name FROM us_500").show()

+----------+
|first_name|
+----------+
|     James|
| Josephine|
|       Art|
|     Lenna|
|   Donette|
|    Simona|
|    Mitsue|
|     Leota|
|      Sage|
|      Kris|
|     Minna|
|      Abel|
|     Kiley|
|  Graciela|
|     Cammy|
|    Mattie|
|   Meaghan|
|    Gladys|
|      Yuki|
|  Fletcher|
+----------+
only showing top 20 rows



In [ ]:
# Perform basic filtering using dataframe
df.filter(df["city"] == "Chicago").show()

+----------+---------+--------------------+--------------------+-------+------+-----+-----+------------+------------+--------------------+--------------------+
|first_name|last_name|        company_name|             address|   city|county|state|  zip|      phone1|      phone2|               email|                 web|
+----------+---------+--------------------+--------------------+-------+------+-----+-----+------------+------------+--------------------+--------------------+
|    Mitsue|  Tollner|  Morlong Associates|           7 Eads St|Chicago|  Cook|   IL|60632|773-573-6914|773-924-8565|mitsue_tollner@ya...|http://www.morlon...|
|      Viva|  Toelkes|   Mark Iv Press Ltd|      4284 Dorigo Ln|Chicago|  Cook|   IL|60647|773-446-5569|773-352-3437|viva.toelkes@gmai...|http://www.markiv...|
|     Marti|  Maybury|Eldridge, Kristin...|4 Warehouse Point...|Chicago|  Cook|   IL|60638|773-775-4522|773-539-1058|marti.maybury@yah...|http://www.eldrid...|
|  Valentin|   Klimek|Schmid, Gayanne K.

In [ ]:
# Perform basic filtering using sql

spark.sql("SELECT first_name, city  FROM us_500 WHERE city='Chicago' ").show()

+----------+-------+
|first_name|   city|
+----------+-------+
|    Mitsue|Chicago|
|      Viva|Chicago|
|     Marti|Chicago|
|  Valentin|Chicago|
|   Carmela|Chicago|
|     Erick|Chicago|
|  Nichelle|Chicago|
+----------+-------+



In [ ]:
# Perfom basic aggregation of data using dataframe
df.groupBy("city").count().show()

+-------------+-----+
|         city|count|
+-------------+-----+
|    Fairbanks|    2|
|    Worcester|    3|
|      Hanover|    1|
|Bowling Green|    1|
|   Harrisburg|    1|
|     Palatine|    1|
|     Harrison|    1|
|      Phoenix|    5|
|   Plainfield|    1|
|  Cherry Hill|    2|
|  Cedar Grove|    1|
|   Perrysburg|    1|
|     Brighton|    1|
|   Toms River|    1|
|     Lynbrook|    1|
|        Omaha|    1|
|    Anchorage|    4|
|    Hampstead|    1|
|      Bothell|    1|
|      Anaheim|    1|
+-------------+-----+
only showing top 20 rows



In [ ]:
# Perfom basic aggregation of data using sql
spark.sql("SELECT city, COUNT(city) as count FROM us_500 GROUP BY city").show()

+-------------+-----+
|         city|count|
+-------------+-----+
|    Fairbanks|    2|
|    Worcester|    3|
|      Hanover|    1|
|Bowling Green|    1|
|   Harrisburg|    1|
|     Palatine|    1|
|     Harrison|    1|
|      Phoenix|    5|
|   Plainfield|    1|
|  Cherry Hill|    2|
|  Cedar Grove|    1|
|   Perrysburg|    1|
|     Brighton|    1|
|   Toms River|    1|
|     Lynbrook|    1|
|        Omaha|    1|
|    Anchorage|    4|
|    Hampstead|    1|
|      Bothell|    1|
|      Anaheim|    1|
+-------------+-----+
only showing top 20 rows



----


## Authors


[Mehran Morabbi Pazoki](https://www.linkedin.com/in/mehran-pazoki-6a3372175/)

